# Обязательная часть

## Скачивание файлов

In [ ]:
! sudo apt install clustalw bedtools proteinortho

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO
import os

In [98]:
base = "http://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/"
paths = {
    "E_billingiae": "000/196/615/GCF_000196615.1_ASM19661v1",
    "E_gerundensis": "020/342/335/GCF_020342335.1_ASM2034233v1",
    "E_persicina": "019/844/095/GCF_019844095.1_ASM1984409v1",
    "E_pyrifoliae": "002/952/315/GCF_002952315.1_ASM295231v1",
    "E_tasmaniensis": "000/026/185/GCF_000026185.1_ASM2618v1"
}
genomic_end = "_genomic.fna.gz"
feature_end = "_feature_table.txt.gz"
protein_end = "_protein.faa.gz"

! mkdir -p genomes histograms zhunt_result TSS intersect ortho clusters/raw clusters/aligned pictures

In [ ]:
for key in paths.keys():
    os.system(f"wget -c -nv {base}{paths[key]}/{paths[key].split('/')[-1]}{genomic_end} -O 'genomes/{key}{genomic_end}'")
    os.system(f"wget -c -nv {base}{paths[key]}/{paths[key].split('/')[-1]}{feature_end} -O 'genomes/{key}{feature_end}'")
    os.system(f"gzip -f -d genomes/{key}{genomic_end}")
    os.system(f"gzip -f -d genomes/{key}{feature_end}")

In [ ]:
for key in paths.keys():
    os.system(f"wget -c -nv {base}{paths[key]}/{paths[key].split('/')[-1]}{protein_end} -O 'genomes/{key}{protein_end}'")
    os.system(f"gzip -f -d genomes/{key}{protein_end}")

## Анализ аннотированных геномов

In [3]:
genomes_list = list(paths.keys())

In [112]:
def annot_analyse(genome):
    features = pd.read_csv(f"genomes/{genome}_feature_table.txt", sep='\t')
    features = features.loc[
        (features['# feature']=='gene') & 
        (features['seq_type']=='chromosome')
    ]
    seqs = [seq_record for seq_record in SeqIO.parse(f"genomes/{genome}_genomic.fna", "fasta")]
    # Choose complete genome, remove plasmids

    genome_len = max(list(map(lambda x: len(x.seq), seqs)))
    total_len = sum(list(map(lambda x: len(x.seq), seqs)))

    total_genes_len = features["feature_interval_length"].sum()

    return [len(features), round(total_genes_len/genome_len*100, 2), genome_len, total_len, len(seqs)]


In [113]:
annot_dict = dict()
for genome in genomes_list:
    annot_dict[genome] = annot_analyse(genome)

pd.DataFrame(annot_dict, 
index=["Genes number", "Genome coverage", 
"Genome length", "Genome + plasmid length", "Number of sequences"]
)

,E_billingiae,E_gerundensis,E_persicina,E_pyrifoliae,E_tasmaniensis
Genes number,4703.00,3458.00,4311.00,3746.00,3569.00
Genome coverage,88.99,88.19,88.61,86.27,86.71
Genome length,5100167.00,3748909.00,4664605.00,4027225.00,3883467.00
Genome + plasmid length,5372268.00,4437416.00,4802925.00,4075681.00,4067864.00
Number of sequences,3.00,3.00,2.00,2.00,6.00


## Предсказание участков Z-DNA

In [ ]:
os.system("wget -c -nv https://raw.githubusercontent.com/vanya-antonov/hse22-project/main/zhunt3-alan.c")

In [ ]:
os.system("gcc zhunt3-alan.c -lm -o zhunt3")

In [237]:
import os
import subprocess
import tempfile
from pathlib import Path
from subprocess import DEVNULL, PIPE

ZH_EXECUTABLE = Path("../hse22_project/zhunt3")
assert ZH_EXECUTABLE.is_file()

def zhunt(query: str, windowsize: int = 6, minsize: int = 3, maxsize: int = 6):
    assert set(query).issubset({"A", "C", "G", "T", "N"})
    fd, temp = tempfile.mkstemp()
    os.close(fd)
    with open(temp, 'w') as stream:
        stream.write(query)

    subprocess.run(
        [ZH_EXECUTABLE, 
         str(windowsize), str(minsize), str(maxsize), temp],
        check=True, stdout=PIPE, stderr=DEVNULL,
        input=query, encoding='ascii'
    )
    with open(temp + ".Z-SCORE", 'r') as stream:
        df = pd.read_csv(stream,
                         names=['Start', 'End', 'nu-1', 'nu-2', 'nu-3', 
                                'ZH-Score', 'Sequence', 'Conformation'],
                         skiprows=1, sep='\s+')
    os.remove(temp)
    os.remove(temp + ".Z-SCORE")
    df = df.loc[df["ZH-Score"] > 500]
    return df[['Start', 'End', 'ZH-Score', 'Sequence', 'Conformation']]

In [238]:
def choose_max_seq(genome):
    seqs = [str(seq_record.seq) for seq_record in SeqIO.parse(f"genomes/{genome}_genomic.fna", "fasta")]
    # Choose complete genome, remove plasmids
    max_scaffold = max(seqs, key=len)
    return max_scaffold

In [239]:
zhunt_dict = dict()
for genome in genomes_list:
    zhunt_dict[genome] = zhunt(choose_max_seq(genome))

In [242]:
zhunt_dict[genomes_list[0]]

,Start,End,ZH-Score,Sequence,Conformation
733,734,742,766.6232,cgtgcgtg,ASASASAS
1677,1678,1690,1356.2120,gggcgtgcccgc,SASASASASASA
1679,1680,1690,1201.6710,gcgtgcccgc,SASASASASA
1744,1745,1753,516.0707,ttgcgcgc,SASASASA
1746,1747,1753,883.5764,gcgcgc,SASASA
...,...,...,...,...,...
5096951,5096952,5096962,2183.5740,cgtgcgtgtg,ASASASASAS
5097858,5097859,5097871,717.9641,aagcgtatgcgc,SASASASASASA
5097860,5097861,5097871,948.8341,gcgtatgcgc,SASASASASA
5098505,5098506,5098512,883.5764,gcgcgc,SASASA


In [243]:
def process_zhunt_data(df):
    df = df.sort_values(["End", "Start"], ascending=True)

    # Count length with overlapping positions
    pos = 0
    length = 0
    for i, row in df.iterrows():
        if row.Start > pos:
            length += row.End - row.Start + 1
            pos = row.End
        elif row.Start < pos and row.End > pos:
            length += row.End - pos + 1
            pos = row.End
    
    return [len(df), length]

In [244]:
zhunt_stats = dict()
for genome in genomes_list:
    zhunt_stats[genome] = process_zhunt_data(zhunt_dict[genome])

pd.DataFrame(zhunt_stats, index=["Number of Z-DNA regions (with overlaps)", "Z-DNA length"])

,E_billingiae,E_gerundensis,E_persicina,E_pyrifoliae,E_tasmaniensis
Number of Z-DNA regions (with overlaps),29749,33366,25907,25346,26301
Z-DNA length,119602,131756,107026,102084,103424


### Рисуем гистограммы

In [90]:
sns.set_theme()
for genome in genomes_list:
    sns.histplot(data=zhunt_dict[genome], x="ZH-Score", bins=30, log_scale=True)
    plt.title(genome.replace('_', '. ')+" ZH-Scores")
    plt.savefig(f"histograms/{genome}.png", dpi=400)
    plt.close()

### Сохраняем данные и создаём .bed файлы

In [245]:
for genome in genomes_list:
    zhunt_dict[genome].to_csv(f"zhunt_result/{genome}.zhunt.txt", index=None)

In [246]:
for genome in genomes_list:
    df = zhunt_dict[genome].copy()
    bed_df = pd.DataFrame(
        {
            "chrom": [genome] * len(df),
            "chromStart": df.Start,
            "chromEnd": df.End,
            "name": df.index,
            "score": df["ZH-Score"]
        }
    )

    bed_df.to_csv(f"zhunt_result/{genome}.zhunt.bed", index=None, sep='\t')


### Объединяем пересечения

In [247]:
for genome in genomes_list:
    os.system(f"bedtools merge -c 5 -o max -i zhunt_result/{genome}.zhunt.bed > zhunt_result/{genome}.zhunt.merged.bed")

## Ассоциируем предсказанные участки Z-DNA с промоторами генов

In [369]:
features_dict = dict()

for genome in genomes_list:
    features = pd.read_csv(f"genomes/{genome}_feature_table.txt", sep='\t')
    features = features.loc[
        (features['seq_type']=='chromosome')
    ]
    locuses = list(np.unique(features.locus_tag))
    df = features.copy()

    counter = 1
    for locus in locuses:
        prod = list(features['product_accession'].loc[features.locus_tag == locus].dropna())
        if prod == []:
            prod = ["RNA" + str(counter)]
            counter += 1
            df['product_accession'].loc[df.locus_tag == locus] = prod[0]
        elif len(prod) == 1:
            df['product_accession'].loc[df.locus_tag == locus] = prod[0]
        else:
            print(prod)
    df = df.loc[df['# feature'] == 'gene']
    df = df[["# feature", "class" , "start", "end", "strand", "product_accession", "locus_tag"]]
    features_dict[genome] = df

In [370]:
for genome in genomes_list:
    features_dict[genome]['TSS'] = features_dict[genome].apply(lambda x: x.start if x.strand == "+" else x.end, axis=1)

In [250]:
for genome in genomes_list:
    df = features_dict[genome].copy()
    bed_df = pd.DataFrame(
        {
            "chrom": [genome] * len(df),
            "chromStart": df.TSS,
            "chromEnd": df.TSS,
            "name": df.product_accession
        }
    )

    bed_df.to_csv(f"TSS/{genome}.bed", index=None, header=None, sep='\t')

In [251]:
pd.DataFrame(
    {
        "Species": genomes_list,
        "Len": [annot_dict[genome][2] for genome in genomes_list]
    }
).to_csv(f"TSS/my.genomes", index=None, header=None, sep='\t')

In [252]:
for genome in genomes_list:
    os.system(f"bedtools slop -g TSS/my.genomes -i TSS/{genome}.bed -b 100 > TSS/{genome}.slop.bed")
    os.system(f"bedtools intersect -a TSS/{genome}.slop.bed -b zhunt_result/{genome}.zhunt.merged.bed -wb > intersect/{genome}.inter.bed")

## Создаём кластеры

In [117]:
!proteinortho -project=erwinia genomes/*faa
!mv erwinia* ortho/

*****************************************************************
Proteinortho with PoFF version 6.0.14 - An orthology detection tool
*****************************************************************
Detected 8 available CPU threads (adjust this with -cpus), Detected 'diamond' version 0.9.30
Checking input files.
Checking genomes/E_billingiae_protein.faa... ok
Checking genomes/E_gerundensis_protein.faa... ok
Checking genomes/E_persicina_protein.faa... ok
Checking genomes/E_pyrifoliae_protein.faa... ok
Checking genomes/E_tasmaniensis_protein.faa... ok

**Step 1**
Generating indices.
The database for 'genomes/E_billingiae_protein.faa' is present and will be used
The database for 'genomes/E_persicina_protein.faa' is present and will be used
The database for 'genomes/E_gerundensis_protein.faa' is present and will be used
The database for 'genomes/E_tasmaniensis_protein.faa' is present and will be used
The database for 'genomes/E_pyrifoliae_protein.faa' is present and will be used

**Step 2

### Статистика по кластерам

In [371]:
clusters = pd.read_csv("ortho/erwinia.proteinortho.tsv", sep='\t')
clusters.sort_values("Alg.-Conn.", ascending=False, inplace=True)
clusters.reset_index(drop=True, inplace=True)
clusters.head(5)

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa
0,2,2,1.0,WP_013203652.1,*,*,*,WP_012440702.1
1,2,2,1.0,*,*,*,WP_012669311.1,WP_012440279.1
2,2,2,1.0,*,*,*,WP_012669308.1,WP_012440282.1
3,2,2,1.0,*,*,*,WP_012669307.1,WP_012440283.1
4,2,2,1.0,*,*,*,WP_012669404.1,WP_012440317.1


In [372]:
number_of_clusters = len(clusters)
number_of_clusters

4337

In [133]:
sns.set_theme()
sns.countplot(data=clusters, x="# Species")
plt.title("Distribution of species in clusters")
#plt.xticks(ticks=[2, 3, 4, 5], labels=['2', '3', '4', '5'])
plt.tight_layout()
plt.savefig(f"histograms/clusters.png", dpi=400)
plt.close()

### Пересекаем кластеры и Z-DNA

In [373]:
clusters = pd.read_csv("ortho/erwinia.proteinortho.tsv", sep='\t')
clusters = clusters.loc[clusters['# Species']==5]
clusters.sort_values("Alg.-Conn.", ascending=False, inplace=True)
clusters.reset_index(drop=True, inplace=True)
clusters.head(5)

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa
0,5,5,0.982,WP_013204225.1,WP_067427185.1,WP_222681877.1,WP_015899125.1,WP_012442922.1
1,5,5,0.981,WP_173363027.1,WP_067429596.1,WP_174522410.1,WP_104945051.1,WP_012441756.1
2,5,5,0.981,WP_013201311.1,WP_225700887.1,WP_062748436.1,WP_012668725.1,WP_012442032.1
3,5,5,0.981,WP_013200102.1,WP_067426621.1,WP_062746920.1,WP_012666468.1,WP_012439841.1
4,5,5,0.981,WP_013203471.1,WP_067433095.1,WP_245006712.1,WP_012667256.1,WP_193345526.1


In [374]:
def read_zdna_df(genome):
    zdna = pd.read_csv(f"intersect/{genome}.inter.bed", sep='\t', 
                names=["Sp1", "St1", "En1", "Protein", "Sp2", "Start", "End", f"Score_{genome}"]
                )[["Protein", f"Score_{genome}", "Start", "End"]]
    zdna.sort_values(f"Score_{genome}", ascending=False, inplace=True)
    # Drop duplicate proteins, keep maximal score
    zdna.drop_duplicates(subset='Protein', inplace=True)
    zdna.set_index("Protein", inplace=True)
    return zdna

In [375]:
for genome in genomes_list:
    zdna = read_zdna_df(genome)[f"Score_{genome}"]
    list_of_zhscores = [float(zdna.loc[prot]) if prot in list(zdna.index) else None for prot in clusters[f"{genome}_protein.faa"]]
    clusters[f"{genome}_protein.score"] = list_of_zhscores
clusters.head()

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa,E_billingiae_protein.score,E_gerundensis_protein.score,E_persicina_protein.score,E_pyrifoliae_protein.score,E_tasmaniensis_protein.score
0,5,5,0.982,WP_013204225.1,WP_067427185.1,WP_222681877.1,WP_015899125.1,WP_012442922.1,NaN,28780.500,766.6232,NaN,NaN
1,5,5,0.981,WP_173363027.1,WP_067429596.1,WP_174522410.1,WP_104945051.1,WP_012441756.1,NaN,2943.461,2943.4610,NaN,1790.574
2,5,5,0.981,WP_013201311.1,WP_225700887.1,WP_062748436.1,WP_012668725.1,WP_012442032.1,908.3955,NaN,NaN,NaN,NaN
3,5,5,0.981,WP_013200102.1,WP_067426621.1,WP_062746920.1,WP_012666468.1,WP_012439841.1,NaN,1733.503,NaN,783.823,865.283
4,5,5,0.981,WP_013203471.1,WP_067433095.1,WP_245006712.1,WP_012667256.1,WP_193345526.1,NaN,NaN,NaN,NaN,NaN


In [ ]:
chosen = clusters.dropna(subset=clusters.columns[-5:])
means = chosen.iloc[:, 8:13].mean(axis=1)
chosen["mean_score"] = means
chosen.sort_values("mean_score", ascending=False, inplace=True)
chosen = chosen.iloc[:8,:]

In [377]:
chosen

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa,E_billingiae_protein.score,E_gerundensis_protein.score,E_persicina_protein.score,E_pyrifoliae_protein.score,E_tasmaniensis_protein.score,mean_score
927,5,5,0.936,WP_013201010.1,WP_067428338.1,WP_118664543.1,WP_012668911.1,WP_012442250.1,138924.1000,28780.500,13713.9900,138924.1000,138924.1000,91853.35800
531,5,5,0.950,WP_013200749.1,WP_067427946.1,WP_062747519.1,WP_012667092.1,WP_012440496.1,3700.9720,65884.110,4876.7350,302785.5000,21732.3800,79795.93940
1306,5,5,0.922,WP_041691924.1,WP_225700936.1,WP_062747774.1,WP_014539290.1,WP_042959064.1,3428.5290,4263.555,66470.7800,198956.2000,27872.6600,60198.34480
230,5,5,0.961,WP_013201926.1,WP_225700696.1,WP_062743475.1,WP_012668206.1,WP_012441548.1,826.8355,904.320,883.5764,211093.9000,883.5764,42918.44166
701,5,5,0.944,WP_013204048.1,WP_209500727.1,WP_062744481.1,WP_012666707.1,WP_012440081.1,138924.1000,65884.110,3661.1100,883.5764,883.5764,42047.29456
41,5,5,0.973,WP_013201017.1,WP_225700938.1,WP_118664541.1,WP_012668905.1,WP_012442243.1,13713.9900,3993.322,138924.1000,883.5764,883.5764,31679.71296
268,5,5,0.959,WP_013200502.1,WP_067433954.1,WP_062745643.1,WP_004160101.1,WP_012442652.1,1090.5640,38833.580,38833.5800,38833.5800,38833.5800,31284.97680
974,5,5,0.935,WP_013200413.1,WP_067434130.1,WP_062748062.1,WP_014539549.1,WP_012442757.1,28780.5000,27872.660,27872.6600,27872.6600,27872.6600,28054.22800


### Выравниваем белки

In [281]:
aa_seqs = dict()
for genome in genomes_list:
    aa_seqs[genome] = SeqIO.to_dict(SeqIO.parse(f"genomes/{genome}_protein.faa", "fasta"))


In [282]:
counter = 1

for i, row in chosen.iterrows():
    seqs_in_cluster = []
    for genome in genomes_list:
        protein_id = row[f"{genome}_protein.faa"]
        seqs_in_cluster.append(aa_seqs[genome][protein_id])
    SeqIO.write(seqs_in_cluster, f"clusters/raw/cluster_{counter}.faa", "fasta")
    counter += 1

In [380]:
for cl in range(1,9):
    os.system(f"clustalw -align -type=PROTEIN -infile='clusters/raw/cluster_{cl}.faa' -OUTFILE='clusters/aligned/cluster_{cl}.aln'")
! rm -rf clusters/raw/*.dnd

### Визуализируем

In [284]:
from dna_features_viewer import GraphicFeature, GraphicRecord

In [378]:
zdna_dict = dict()
for genome in genomes_list:
    features_dict[genome].set_index("product_accession", drop=False, inplace=True)
    zdna_dict[genome] = read_zdna_df(genome)

In [ ]:
counter = 1
for k, row in chosen.iterrows():
    fig, axes = plt.subplots(nrows=5, ncols=1,figsize=(12, 6))
    fig.suptitle(f'Cluster {counter}')

    for i, genome in enumerate(genomes_list):
        zdna = zdna_dict[genome]
        features = features_dict[genome]

        prot = row.loc[genome + '_protein.faa']
        Z_DNA_label = "Z-DNA, ZH-Score=" + str(round(zdna.loc[prot][f"Score_{genome}"]))
        coords = [zdna.loc[prot, "Start"], zdna.loc[prot, "End"],
                  features.loc[prot, "start"], features.loc[prot, "end"]]
        minimum = min(coords)

        strand = +1 if features.loc[prot, "strand"]=="+" else -1

        features=[
        GraphicFeature(start=coords[0]-minimum+26, end=coords[1]-minimum+26, 
                   strand=+1, color="#0057b8", label=Z_DNA_label),
        GraphicFeature(start=coords[2]-minimum+26, end=coords[3]-minimum+26, 
                   strand=strand, color="#ffd700", label=genome + ": " + prot),
                   ]
        record = GraphicRecord(sequence_length=max(coords)-minimum+50, features=features)
        record.plot(ax=axes[i])
    plt.savefig(f'pictures/Cluster_{counter}.png', dpi=500)
    counter += 1

# Бонусная часть

## Поиск G-квадруплексов

In [ ]:
na_seqs = dict()
for genome in genomes_list:
    na_seqs[genome] = choose_max_seq(genome)

### Ставим PQSfinder и запускаем

In [ ]:
! Rscript configure.r

В `pqsfinder.r` немного ослабили ограничения

In [328]:
import os
import subprocess
import tempfile
from pathlib import Path
from subprocess import DEVNULL, PIPE

Q_EXECUTABLE = Path("pqsfinder.r")
assert Q_EXECUTABLE.is_file()


def pqs_exec(query: str):
    assert set(query).issubset({"A", "C", "G", "T", "N"})
    fd, temp = tempfile.mkstemp()
    os.close(fd)
    with open(temp, 'w') as stream:
        stream.write(">chr\n"+query)

    subprocess.run(
        ["Rscript", Q_EXECUTABLE, temp, temp + ".qgrs"],
        check=True, stdout=PIPE, stderr=DEVNULL,
        input=query, encoding='ascii'
    )
    
    with open(temp + ".qgrs", 'r') as stream:
        df = pd.read_csv(stream,
                         #names=['ID', 'T1', 'T2', 'T3', 'T4', 'TS', 'GS', 'SEQ'],
                         skiprows=0, sep=' ').iloc[:,:8]
    os.remove(temp)
    os.remove(temp + ".qgrs")
    #df = df.loc[df["score"] > 500]
    #return df
    return df[['start', 'end', 'score', 'strand']]

In [327]:
pqs_dict = dict()
for genome in genomes_list:
    pqs_dict[genome] = pqs_exec(na_seqs[genome])

In [329]:
def process_pqs_data(df):
    df = df.sort_values(["end", "start"], ascending=True)

    # Count length with overlapping positions
    pos = 0
    length = 0
    for i, row in df.iterrows():
        if row.start > pos:
            length += row.end - row.start + 1
            pos = row.end
        elif row.start < pos and row.end > pos:
            length += row.end - pos + 1
            pos = row.end
    
    return [len(df), length]

In [330]:
pqs_stats = dict()
for genome in genomes_list:
    pqs_stats[genome] = process_pqs_data(pqs_dict[genome])

pd.DataFrame(pqs_stats, index=["Number of quadruplex regions (with overlaps)", "Quadruplex length"])

,E_billingiae,E_gerundensis,E_persicina,E_pyrifoliae,E_tasmaniensis
Number of quadruplex regions (with overlaps),445,212,913,445,508
Quadruplex length,12914,6138,27800,13259,15058


### Рисуем гистограммы

In [331]:
sns.set_theme()
for genome in genomes_list:
    sns.histplot(data=pqs_dict[genome], x="score", bins=30, log_scale=False)
    plt.title(genome.replace('_', '. ')+" Quadruplex Scores")
    plt.savefig(f"histograms/{genome}_quadruplex.png", dpi=400)
    plt.close()

### Сохраняем данные и создаём .bed файлы

In [332]:
! mkdir -p pqs_result
for genome in genomes_list:
    pqs_dict[genome].to_csv(f"pqs_result/{genome}.pqs.txt", index=None)

In [333]:
for genome in genomes_list:
    df = pqs_dict[genome].copy()
    bed_df = pd.DataFrame(
        {
            "chrom": [genome] * len(df),
            "chromStart": df.start,
            "chromEnd": df.end,
            "name": df.index,
            "score": df.score,
            "strand": df.strand
        }
    )

    bed_df.to_csv(f"pqs_result/{genome}.pqs.bed", index=None, sep='\t')


### Объединяем пересечения

In [334]:
for genome in genomes_list:
    os.system(f"bedtools merge -c 5,6 -o max,distinct -i pqs_result/{genome}.pqs.bed > pqs_result/{genome}.pqs.merged.bed")

### Ассоциируем предсказанные участки квадруплексов с промоторами генов

In [335]:
for genome in genomes_list:
    os.system(f"bedtools intersect -a TSS/{genome}.slop.bed -b pqs_result/{genome}.pqs.merged.bed -wb > intersect/{genome}_quadruplex.inter.bed")

In [336]:
def read_pqs_df(genome):
    pqs = pd.read_csv(f"intersect/{genome}_quadruplex.inter.bed", sep='\t', 
                names=["Sp1", "St1", "En1", "Protein", "Sp2", "Start", "End", f"Score_{genome}", "Strand"]
                )[["Protein", f"Score_{genome}", "Start", "End", "Strand"]]
    pqs.sort_values(f"Score_{genome}", ascending=False, inplace=True)
    # Drop duplicate proteins, keep maximal score
    pqs.drop_duplicates(subset='Protein', inplace=True)
    pqs.set_index("Protein", inplace=True)
    return pqs

In [337]:
for genome in genomes_list:
    pqs = read_pqs_df(genome)[f"Score_{genome}"]
    list_of_scores = [float(pqs.loc[prot]) if prot in list(pqs.index) else None for prot in clusters[f"{genome}_protein.faa"]]
    clusters[f"{genome}_protein.quadro"] = list_of_scores
clusters.head()

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa,E_billingiae_protein.score,E_gerundensis_protein.score,E_persicina_protein.score,E_pyrifoliae_protein.score,E_tasmaniensis_protein.score,E_billingiae_protein.quadro,E_gerundensis_protein.quadro,E_persicina_protein.quadro,E_pyrifoliae_protein.quadro,E_tasmaniensis_protein.quadro
0,5,5,0.982,WP_013204225.1,WP_067427185.1,WP_222681877.1,WP_015899125.1,WP_012442922.1,NaN,28780.500,766.6232,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,5,0.981,WP_173363027.1,WP_067429596.1,WP_174522410.1,WP_104945051.1,WP_012441756.1,NaN,2943.461,2943.4610,NaN,1790.574,NaN,NaN,NaN,NaN,NaN
2,5,5,0.981,WP_013201311.1,WP_225700887.1,WP_062748436.1,WP_012668725.1,WP_012442032.1,908.3955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,5,0.981,WP_013200102.1,WP_067426621.1,WP_062746920.1,WP_012666468.1,WP_012439841.1,NaN,1733.503,NaN,783.823,865.283,NaN,NaN,NaN,NaN,NaN
4,5,5,0.981,WP_013203471.1,WP_067433095.1,WP_245006712.1,WP_012667256.1,WP_193345526.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [339]:
chosen = clusters.dropna(subset=clusters.columns[-5:], how="all", thresh=4)
chosen

,# Species,Genes,Alg.-Conn.,E_billingiae_protein.faa,E_gerundensis_protein.faa,E_persicina_protein.faa,E_pyrifoliae_protein.faa,E_tasmaniensis_protein.faa,E_billingiae_protein.score,E_gerundensis_protein.score,E_persicina_protein.score,E_pyrifoliae_protein.score,E_tasmaniensis_protein.score,E_billingiae_protein.quadro,E_gerundensis_protein.quadro,E_persicina_protein.quadro,E_pyrifoliae_protein.quadro,E_tasmaniensis_protein.quadro
1143,5,5,0.929,WP_013200491.1,WP_225701318.1,WP_062745668.1,WP_012666867.1,WP_012442663.1,612.3848,NaN,612.3848,3428.5290,3428.529,47.0,48.0,55.0,76.0,76.0
1622,5,5,0.897,WP_013203132.1,WP_187485638.1,WP_062742990.1,WP_104945108.1,WP_012440908.1,NaN,NaN,883.5764,NaN,NaN,57.0,NaN,53.0,49.0,52.0
2188,5,5,0.298,WP_013203557.1,WP_067432667.1,WP_062744220.1,WP_012669075.1,WP_012442422.1,NaN,13713.99,NaN,880.2792,NaN,59.0,NaN,53.0,97.0,91.0


### Визуализируем

In [354]:
from dna_features_viewer import GraphicFeature, GraphicRecord
quadro_dict = dict()
for genome in genomes_list:
    features_dict[genome].set_index("product_accession", drop=False, inplace=True)
    quadro_dict[genome] = read_pqs_df(genome)

In [ ]:
counter = 1
for k, row in chosen.iterrows():
    rows_num = len(row.iloc[-5:].dropna())
    fig, axes = plt.subplots(nrows=rows_num, ncols=1,figsize=(12, 6*rows_num/5))
    fig.suptitle(f'Cluster {counter}')
    axes_count = 0

    for i, genome in enumerate(genomes_list):
        quad = quadro_dict[genome]
        features = features_dict[genome]

        prot = row.loc[genome + '_protein.faa']
        if not prot in quad.index:
            continue

        quad_label = "Quadruplex, Score=" + str(round(quad.loc[prot][f"Score_{genome}"]))
        coords = [quad.loc[prot, "Start"], quad.loc[prot, "End"],
                  features.loc[prot, "start"], features.loc[prot, "end"]]
        minimum = min(coords)

        strand_q = +1 if quad.loc[prot, "Strand"]=="+" else -1
        strand_f = +1 if features.loc[prot, "strand"]=="+" else -1


        features=[
        GraphicFeature(start=coords[0]-minimum+26, end=coords[1]-minimum+26, 
                   strand=strand_q, color="#0057b8", label=quad_label),
        GraphicFeature(start=coords[2]-minimum+26, end=coords[3]-minimum+26, 
                   strand=strand_f, color="#ffd700", label=genome + ": " + prot),
                   ]
        record = GraphicRecord(sequence_length=max(coords)-minimum+50, features=features)
        record.plot(ax=axes[axes_count])
        axes_count += 1
    plt.savefig(f'pictures/Cluster_{counter}_quadruplex.png', dpi=500)
    counter += 1